 ## Calculate overlap
Generate the DICE coefficient for each segmentation using the *EDIT1 file as reference. 

In [1]:
# Import packages
import ants
import numpy as np
import pandas as pd
from pathlib import Path
import logging
import os
import subprocess

PATH_TO_DATA = Path('test-data')

In [2]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

In [3]:
def refine_list(lst):
    result_list = str(lst).split(',')
    result_list[-1] = result_list[-1].replace("\\n", "").replace("'", "")
    result_list = [float(result_list[4]), float(result_list[5])]
#     result_list = [float(e) for e in result_list]
    
    return(result_list)
    

Before running the next step, I reoriented the reference segmentations using `organizeAndReorient.sh`.

In [7]:
# Run the DICE calculations
df_list = [] # each output is a pandas DataFrame that we'll combine at the end based on this list
for subj in PATH_TO_DATA.glob('*S*'): # e.g., 011_S_6303
    log.info('Processing subject %s', subj.name)
    for ses in subj.glob('20*'): # 2017, 2018, etc.
        log.info('Processing session %s', ses.name)
        
        new_seg_file = str(list(ses.glob("*WMH_seg*"))[0])
        new_10_seg_file = str(list(ses.glob("*WMH_10x_seg*"))[0])
        baseline_seg_file = str(list(ses.glob("*reorient_EDIT1*"))[0])
                
        # copy header from new seg to old seg (only have to do once) to match origins
        cmd = 'c3d {0} {1} -copy-transform -o {1}'.format(new_seg_file, baseline_seg_file)
        log.info(os.system(cmd))
        
        # calculate DICE with c3d
        cmd2="c3d {} {} -overlap 1".format(new_seg_file, baseline_seg_file)
        result = refine_list(subprocess.check_output(cmd2, shell=True))
        cmd3="c3d {} {} -overlap 1".format(new_10_seg_file, baseline_seg_file)
        result10 = refine_list(subprocess.check_output(cmd3, shell=True))
        log.info("DICE, JACOBIAN for both: {} ; {}".format(result, result10))
        
        # calculate label volume with FSL
        cmd4="fslstats {} -V".format(new_seg_file) # volume of non-zero voxels
        result_fsl = float(str(subprocess.check_output(cmd4,shell=True)).split(' ')[1].replace('\n', ''))
        cmd5="fslstats {} -V".format(new_10_seg_file)
        result_10_fsl = float(str(subprocess.check_output(cmd5,shell=True)).split(' ')[1].replace('\n', ''))
        log.info("LABEL VOLUMES: {}; {}".format(result_fsl, result_10_fsl))
        
        # Do the ANTs calculation for both segmentations
        new_seg = ants.image_read(new_seg_file)  # read the data
        new_10_seg = ants.image_read(new_10_seg_file)
        baseline_seg = ants.image_read(baseline_seg_file)
        
        std_comp = ants.label_overlap_measures(baseline_seg, new_seg) # run the calculation
        x10_comp = ants.label_overlap_measures(baseline_seg, new_10_seg)
        
        # Edit the dataframe a little
        std_comp['c3dDice'] = result[0] # Dice
        x10_comp['c3dDice'] = result10[0] 
        std_comp['c3dJacob'] = result[1] # Jacobian
        x10_comp['c3dJacob'] = result10[1]
        std_comp['volume'] = result_fsl
        x10_comp['volume'] = result_10_fsl
        std_comp.insert(0, 'SubjectID', subj.name) # insert subject name in first position
        x10_comp.insert(0, 'SubjectID', subj.name)
        std_comp.insert(1, 'Type', 'normal') # specify which image is which
        x10_comp.insert(1, 'Type', 'scaled10x')
        std_comp=std_comp.iloc[:1] # just get the first row
        x10_comp=x10_comp.iloc[:1]
        
        df_list.append(std_comp) # add to list for concatenation later
        df_list.append(x10_comp)    
        
df = pd.concat(df_list)
df = df.reset_index(drop=True)
df = df.drop('Label',1) # drop superfluous column
log.info('DONE')

2021-09-22 13:48:20,796 INFO Processing subject 027_S_6034
2021-09-22 13:48:20,796 INFO Processing session 2019-07-24
2021-09-22 13:48:21,477 INFO 0
2021-09-22 13:48:22,408 INFO DICE, JACOBIAN for both: [0.433333, 0.276596] ; [0.464, 0.302083]
2021-09-22 13:48:22,836 INFO LABEL VOLUMES: 872.40332; 770.402954
2021-09-22 13:48:23,544 INFO Processing subject 168_S_6426
2021-09-22 13:48:23,545 INFO Processing session 2018-06-29
2021-09-22 13:48:24,134 INFO 0
2021-09-22 13:48:24,957 INFO DICE, JACOBIAN for both: [0.7808, 0.640421] ; [0.797701, 0.66348]
2021-09-22 13:48:25,340 INFO LABEL VOLUMES: 8552.418945; 8994.019531
2021-09-22 13:48:25,967 INFO Processing subject 027_S_5277
2021-09-22 13:48:25,968 INFO Processing session 2017-11-30
2021-09-22 13:48:26,634 INFO 0
2021-09-22 13:48:27,560 INFO DICE, JACOBIAN for both: [0.435734, 0.278555] ; [0.451092, 0.291232]
2021-09-22 13:48:27,988 INFO LABEL VOLUMES: 1961.994995; 1856.395264
2021-09-22 13:48:28,698 INFO Processing subject 114_S_6347
20

In [8]:
df.to_csv('overlap_data.csv',index=False)